# Machine Learning (CS-433)
## Class Project 1
Simon Canales, Jordan Willemin, Claudio Loureiro

In [2]:
import numpy as np

from proj1_helpers import *
from implementations import *
from utils import *

In [4]:
training_data, labels, ids_tr = load_csv_data("../data/train.csv")
testing_data, labels, ids_te = load_csv_data("../data/test.csv")

In [5]:
# Remove the 999
# TODO

# Least Squares Gradient Descent